In [6]:
import numpy as np  # 导入NumPy
import pandas as pd  # 导入Pandas

df_train = pd.read_csv('./dataset/exoTrain.csv')  # 导入训练集
df_test = pd.read_csv('./dataset/exoTest.csv')  # 导入测试集
print(df_train.head())  # 输入头几行数据
print(df_train.info())  # 输出训练集信息

   LABEL   FLUX.1   FLUX.2   FLUX.3   FLUX.4   FLUX.5   FLUX.6  FLUX.7  \
0      2    93.85    83.81    20.10   -26.98   -39.56  -124.71 -135.18   
1      2   -38.88   -33.83   -58.54   -40.09   -79.31   -72.81  -86.55   
2      2   532.64   535.92   513.73   496.92   456.45   466.00  464.50   
3      2   326.52   347.39   302.35   298.13   317.74   312.70  322.33   
4      2 -1107.21 -1112.59 -1118.95 -1095.10 -1057.55 -1034.48 -998.34   

    FLUX.8  FLUX.9  ...  FLUX.3188  FLUX.3189  FLUX.3190  FLUX.3191  \
0   -96.27  -79.89  ...     -78.07    -102.15    -102.15      25.13   
1   -85.33  -83.97  ...      -3.28     -32.21     -32.21     -24.89   
2   486.39  436.56  ...     -71.69      13.31      13.31     -29.89   
3   311.31  312.42  ...       5.71      -3.73      -3.73      30.05   
4 -1022.71 -989.57  ...    -594.37    -401.66    -401.66    -357.24   

   FLUX.3192  FLUX.3193  FLUX.3194  FLUX.3195  FLUX.3196  FLUX.3197  
0      48.57      92.54      39.32      61.42       5.08  

In [7]:
from sklearn.utils import shuffle  # 导入乱序工具

df_train = shuffle(df_train)  # 乱序训练集
df_test = shuffle(df_test)  # 乱序测试集

In [8]:
X_train = df_train.iloc[:, 1:].values  # 构建特征集（训练）
y_train = df_train.iloc[:, 0].values  # 构建标签集（训练）
X_test = df_test.iloc[:, 1:].values  # 构建特征集（测试）
y_test = df_test.iloc[:, 0].values  # 构建标签集（测试）
y_train = y_train - 1  # 标签转换成惯用的(0，1)分类
y_test = y_test - 1  # 标签转换成惯用的(0，1)分类
print(X_train)  # 打印训练集中的特征
print(y_train)  # 打印训练集中的标签

[[  -45.16    80.56   105.4  ...   653.15   492.87   400.84]
 [ -201.18  -140.13  -153.79 ...    26.31    43.81    50.78]
 [ -900.44  -865.84  -857.47 ... -2031.97 -2078.65 -2221.8 ]
 ...
 [  -48.69   -58.42   -36.2  ...    -4.4    -20.36   -11.43]
 [  976.98   922.62   920.29 ...   361.61   352.63   335.63]
 [   19.16    -4.73    15.58 ...   -58.54   -18.39   -69.87]]
[0 0 0 ... 0 0 0]


In [9]:
X_train = np.expand_dims(X_train, axis=2)  # 张量升阶，以满足序列数据集的要求
X_test = np.expand_dims(X_test, axis=2)  # 张量升阶，以满足序列数据集的要求
print(X_train.shape)

(5087, 3197, 1)


In [11]:
from keras.models import Sequential  # 导入序贯模型
from keras import layers  # 导入所有类型的层
from keras.optimizers import Adam  # 导入优化器

model = Sequential()  # 序贯模型
model.add(layers.Conv1D(32, kernel_size=10, strides=4,
                        input_shape=(3197, 1)))  # 1D CNN层
model.add(layers.MaxPooling1D(pool_size=4, strides=2))  # 池化层
model.add(layers.GRU(256, return_sequences=True))  # 关键，GRU层够要大
model.add(layers.Flatten())  # 展平
model.add(layers.Dropout(0.5))  # Dropout层
model.add(layers.BatchNormalization())  # 批标准化
model.add(layers.Dense(1, activation='sigmoid'))  # 分类输出层
opt = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)  # 设置优化器
model.compile(optimizer=opt,  # 优化器
              loss='binary_crossentropy',  # 交叉熵
              metrics=['accuracy'])  # 准确率

2023-07-02 13:12:08.925394: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-02 13:12:08.927796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-02 13:12:08.937977: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [12]:
history = model.fit(X_train, y_train,  # 训练集
                    validation_split=0.2,  # 部分训练集数据拆分成验证集
                    batch_size=128,  # 批量大小
                    epochs=4,  # 训练轮次
                    shuffle=True)  # 乱序

Epoch 1/4


2023-07-02 13:14:03.062809: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-02 13:14:03.064492: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-02 13:14:03.066212: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

32/32 [==============================] - ETA: 0s - loss: 0.6507 - accuracy: 0.6638

2023-07-02 13:14:42.345914: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-02 13:14:42.346924: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-02 13:14:42.348113: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

32/32 [==============================] - 42s 1s/step - loss: 0.6507 - accuracy: 0.6638 - val_loss: 0.2467 - val_accuracy: 0.9853
Epoch 2/4
32/32 [==============================] - 39s 1s/step - loss: 0.3514 - accuracy: 0.8754 - val_loss: 0.1860 - val_accuracy: 0.9872
Epoch 3/4
32/32 [==============================] - 37s 1s/step - loss: 0.2344 - accuracy: 0.9363 - val_loss: 0.1307 - val_accuracy: 0.9892
Epoch 4/4
32/32 [==============================] - 37s 1s/step - loss: 0.1674 - accuracy: 0.9658 - val_loss: 0.0957 - val_accuracy: 0.9892


In [13]:
from sklearn.metrics import classification_report  # 分类报告
from sklearn.metrics import confusion_matrix  # 混淆矩阵

y_prob = model.predict(X_test)  # 对测试集进行预测
y_pred = np.where(y_prob > 0.5, 1, 0)  #将概率值转换成真值
cm = confusion_matrix(y_pred, y_test)
print('Confusion matrix:\n', cm, '\n')
print(classification_report(y_pred, y_test))

2023-07-02 13:19:06.787013: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-02 13:19:06.788589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-02 13:19:06.790347: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

18/18 [==============================] - 3s 112ms/step
Confusion matrix:
 [[565   5]
 [  0   0]] 

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       570
           1       0.00      0.00      0.00         0

    accuracy                           0.99       570
   macro avg       0.50      0.50      0.50       570
weighted avg       1.00      0.99      1.00       570



/Users/cag2050/Documents/PycharmProjects/zero_basic_machine_learning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/cag2050/Documents/PycharmProjects/zero_basic_machine_learning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/cag2050/Documents/PycharmProjects/zero_basic_machine_learning/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_div

In [14]:
y_pred = np.where(y_prob > 0.15, 1, 0)  # 进行阈值调整
cm = confusion_matrix(y_pred, y_test)
print('Confusion matrix:\n', cm, '\n')
print(classification_report(y_pred, y_test))

Confusion matrix:
 [[553   4]
 [ 12   1]] 

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       557
           1       0.20      0.08      0.11        13

    accuracy                           0.97       570
   macro avg       0.59      0.53      0.55       570
weighted avg       0.96      0.97      0.97       570

